<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/02_elementary_Japanese_NLP_in_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02 日本語のテキストデータの扱い方

* 日本語は、言語の性質上、文が空白によって単語へ分たれていない。
 * 同じように、文が空白によって単語へ分たれていない言語として、他にどのような言語があるか？
* そのため、まず最初に文を単語へ分割する必要がある。
* この作業を**形態素解析**と言う。
 * 長い文字列としての言語データを、単語のような細かい単位へ分割することを、一般にtokenizationと言う。
 * tokenizationの単位は、単語のように意味的なまとまりをもつ単位とは限らない。
 *cf. [SentencePiece](https://arxiv.org/pdf/1808.06226.pdf) ... [この解説](https://qiita.com/taku910/items/7e52f1e58d0ea6e7859c)も参照。
* ここでは、spaCyを介してsudachiというツールで形態素解析をおこなう。
 * 形態素解析システムとしては、他には、MeCabやJUMANが有名。

## 注
* "ContextualVersionConflict ..."というエラーが出たら、上部メニューの「ランタイム」から「ランタイムを再起動」をクリックして、ランタイムを再起動してみてください。

## 02-01 spaCyの準備

### spaCyを最新版にアップデートする。

In [ ]:
! pip install -U spacy

     |████████████████████████████████| 10.0MB 5.6MB/s 
     |████████████████████████████████| 2.1MB 44.5MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


### バージョンを確認する。

In [ ]:
! python -m spacy validate

✔ Loaded compatibility table

====================== Installed models (spaCy v2.3.2) ======================
ℹ spaCy installation: /usr/local/lib/python3.6/dist-packages/spacy

TYPE      NAME             MODEL            VERSION                       
package   en-core-web-sm   en_core_web_sm   2.2.5   --> 2.3.1


============================== Install updates ==============================
Use the following commands to update the model packages:
python -m spacy download en_core_web_sm



### 英語の統計モデルもアップデートしておく。
* 今回は英語データは使わないが。

In [ ]:
! python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.1MB 520kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-cp36-none-any.whl size=12047109 sha256=5f27abff5d7fb52ad9ec3811ac5934908097785263ec1fa5f12a25b9eed5b84c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ruiudnf5/wheels/2b/3f/41/f0b92863355c3ba34bb32b37d8a0c662959da0058202094f46
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
! python -m spacy validate

✔ Loaded compatibility table

====================== Installed models (spaCy v2.3.2) ======================
ℹ spaCy installation: /usr/local/lib/python3.6/dist-packages/spacy

TYPE      NAME             MODEL            VERSION                            
package   en-core-web-sm   en_core_web_sm   2.3.1   ✔



### spaCyの日本語の訓練済み統計モデルをダウンロードする
* 「sm」で終わるものは、小規模なモデル。
* 「md」で終わるものが、中規模のモデル。
* 「lg」で終わるものが、大規模なモデル。
* 詳細は、以下を参照。
 * https://explosion.ai/blog/spacy-v2-3
* ダウンロード時間短縮のため、ここでは小規模モデルを使う。

In [ ]:
! python -m spacy download ja_core_news_sm

     |████████████████████████████████| 7.6MB 522kB/s 
     |████████████████████████████████| 71kB 4.2MB/s 
     |████████████████████████████████| 481kB 7.6MB/s 
  Created wheel for ja-core-news-sm: filename=ja_core_news_sm-2.3.2-cp36-none-any.whl size=7572678 sha256=fcbd869a0d9fb05ed3dd423adc0a1fd276882b9d59197bf7d999d38bf77f4e31
  Stored in directory: /tmp/pip-ephem-wheel-cache-0dsavrb4/wheels/a9/90/6f/9fdba619c4403234b2695fd40d77e635a6c80672a6a05920f0
  Created wheel for sudachipy: filename=SudachiPy-0.4.9-cp36-cp36m-linux_x86_64.whl size=870424 sha256=8726e5ae40887ceafbf9bc3d564cff021cee42cd2714d79e3abdc4faae3ad9d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-0dsavrb4/wheels/79/85/b4/bb2fbbf043bc8128a6cbb769604c410dc5ece89c88b7e57de0
  Created wheel for sudachidict-core: filename=SudachiDict_core-20200722-cp36-none-any.whl size=71120193 sha256=4b5d7a27c9836cb4550f20dece7e90916ff0e00ed401ebd996a0c4157243528e
  Stored in directory: /tmp/pip-ephem-wheel-cache-0dsavrb4/wheels/aa

## 02-02 sudachiによる形態素解析を試してみる。
* spaCyの日本語統計モデルをインストールすると、自動的にsudachiがインストールされる。
* そこで、sudachipyコマンドを使って、コマンドラインで形態素解析を試してみる。

In [ ]:
! echo "すもももももももものうち" | sudachipy

すもも	名詞,普通名詞,一般,*,*,*	李
も	助詞,係助詞,*,*,*,*	も
もも	名詞,普通名詞,一般,*,*,*	もも
も	助詞,係助詞,*,*,*,*	も
もも	名詞,普通名詞,一般,*,*,*	もも
の	助詞,格助詞,*,*,*,*	の
うち	名詞,普通名詞,副詞可能,*,*,*	内
EOS


## 02-03 spaCyによる日本語テキスト処理

### 日本語統計モデルのロード

In [ ]:
from spacy.lang.ja import Japanese

# Load SudachiPy with split mode A (default)
nlp = Japanese()

# Load SudachiPy with split mode B
#cfg = {"split_mode": "B"}
#nlp = Japanese(meta={"tokenizer": {"config": cfg}})

### documentを作成

In [ ]:
doc = nlp('これは、やっぱり相当冗長な日本語の文じゃないかなと思っていたのですが。')
print(doc.text)

これは、やっぱり相当冗長な日本語の文じゃないかなと思っていたのですが。


### spaCyからsudachiを利用した形態素解析の実行
* 分かち書きされた単語そのもの以外の、品詞などの情報も表示させている。
 * tagに「形容動詞」は無く、その語幹が「形状詞」とされている。
 * 「じゃない」の「ない」は、tagでは「形容詞」だが、posでは「AUX」である。

In [ ]:
for token in doc:
  print(f'text:{token.text}, pos:{token.pos_}, tag:{token.tag_}, lemma:{token.lemma_}')

text:これ, pos:PRON, tag:代名詞, lemma:これ
text:は, pos:ADP, tag:助詞-係助詞, lemma:は
text:、, pos:PUNCT, tag:補助記号-読点, lemma:、
text:やっぱり, pos:ADV, tag:副詞, lemma:やっぱり
text:相当, pos:ADV, tag:副詞, lemma:相当
text:冗長, pos:ADJ, tag:形状詞-一般, lemma:冗長
text:な, pos:AUX, tag:助動詞, lemma:だ
text:日本, pos:PROPN, tag:名詞-固有名詞-地名-国, lemma:日本
text:語, pos:NOUN, tag:名詞-普通名詞-一般, lemma:語
text:の, pos:ADP, tag:助詞-格助詞, lemma:の
text:文, pos:NOUN, tag:名詞-普通名詞-一般, lemma:文
text:じゃ, pos:AUX, tag:助動詞, lemma:だ
text:ない, pos:AUX, tag:形容詞-非自立可能, lemma:ない
text:か, pos:PART, tag:助詞-終助詞, lemma:か
text:な, pos:PART, tag:助詞-終助詞, lemma:な
text:と, pos:ADP, tag:助詞-格助詞, lemma:と
text:思っ, pos:VERB, tag:動詞-一般, lemma:思う
text:て, pos:SCONJ, tag:助詞-接続助詞, lemma:て
text:い, pos:AUX, tag:動詞-非自立可能, lemma:いる
text:た, pos:AUX, tag:助動詞, lemma:た
text:の, pos:SCONJ, tag:助詞-準体助詞, lemma:の
text:です, pos:AUX, tag:助動詞, lemma:です
text:が, pos:SCONJ, tag:助詞-接続助詞, lemma:が
text:。, pos:PUNCT, tag:補助記号-句点, lemma:。


### 英語の場合（復習）

In [ ]:
import spacy

nlp_en = spacy.load("en_core_web_sm")
doc = nlp_en("Apple is looking at buying U.K. startup for $1 billion.")
print(doc.text)

Apple is looking at buying U.K. startup for $1 billion.


In [ ]:
for token in doc:
  print(f'text:{token.text}, pos:{token.pos_}, tag:{token.tag_}, lemma:{token.lemma_}')

text:Apple, pos:PROPN, tag:NNP, lemma:Apple
text:is, pos:AUX, tag:VBZ, lemma:be
text:looking, pos:VERB, tag:VBG, lemma:look
text:at, pos:ADP, tag:IN, lemma:at
text:buying, pos:VERB, tag:VBG, lemma:buy
text:U.K., pos:PROPN, tag:NNP, lemma:U.K.
text:startup, pos:NOUN, tag:NN, lemma:startup
text:for, pos:ADP, tag:IN, lemma:for
text:$, pos:SYM, tag:$, lemma:$
text:1, pos:NUM, tag:CD, lemma:1
text:billion, pos:NUM, tag:CD, lemma:billion
text:., pos:PUNCT, tag:., lemma:.


# 課題2: 文書のbag-of-words表現を得る

* （この課題は、宿題ではなく、いまここで作業しつつ解いてしまいます。）
* Wikipediaの複数のエントリを、lemmaを半角スペースでつないだ、長い文字列へ変換する。
 * ここでは、コンピュータ科学の様々な分野のエントリを題材として使う。
* scikit-learnのCountVectorizerやTfidfVectorizerを使って、各エントリにおける単語の出現頻度からなる特徴ベクトルを得る。
* **文書を、そこに現れる単語の出現頻度やTF-IDFを使ってベクトル表現している限り、単語の出現順序は無視されている。文書に現れる単語は、バッグに入った品物のように扱われており、各々の品物が何個入っているかだけが問題とされていて、出現順の情報は失われている。このように、文書を単語のmultisetとしてモデル化することを、bag-of-words modelingと呼ぶ。**
* 特徴ベクトルどうしの類似度を計算し、「人工知能」分野と最も似ている順に　３つの分野がどの分野かを求める。
 * 答えは自分の感覚でチェック。
 * 特徴ベクトルを作る時に単語の品詞を名詞に限定するなど、品詞の情報を使うことで結果を改善できるかどうかも、余裕があれば試行錯誤する。

## 課題の手順(1)

### Wikipediaのエントリをダウンロードして形態素解析を適用

* Wikipediaの「人工知能」エントリをダウンロードする。
 * https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD
* そして、そこに含まれる段落（__`<p>`__タグで囲まれた範囲）を列挙する。
* 各段落のテキストに形態素解析を適用する。
* 形態素解析で得られたlemmaを半角スペースでつないで、エントリ全体をひとつの長い文字列にする。
 * text、つまり、単語に分たれたそのままの文字列を半角スペースでつなぐのではない。
 * lemma（原型に戻したもの）を半角スペースでつなぐこと。
 * posやtagを見て、不要そうな単語を適当に削除してもよい。


In [ ]:
# 「人工知能」エントリをダウンロードしてparserのインスタンスを作る。

from bs4 import BeautifulSoup
from urllib.request import urlopen

url = 'https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# 段落のテキストを取得する。

lines = list()
for para in soup.find_all('p'):
  lines.append(para.text)

In [ ]:
lines

In [ ]:
# Sudachiで形態素解析し、分かち書き後のlemmaを取得する。

x_pos = ['SPACE', 'PUNCT', 'AUX', 'ADP', 'SYM', 'DET', 'SCONJ', 'PART']
tokens = list()
for line in lines:
  for token in nlp(line):
    pos = token.pos_
    if pos not in x_pos:
      #print(f'text:{token.text}, pos:{token.pos_}, tag:{token.tag_}, lemma:{token.lemma_}')
      tokens.append(token.lemma_)

In [ ]:
# すべてのlemmaを半角スペースでつないで、長い文字列にする。

doc_AI = ' '.join(tokens)
print(doc_AI)

人工 知能 じん こうち 英 artificial intelligence AI エーアイ 計算 computation いう 概念 コンピュータ Computer いう 道具 用いる 知能 研究 計算 機 科学 computer science 一 分野 指す 語 1 言語 理解 推論 問題 解決 知的 行動 人間 代わる コンピューター 行う 技術 2 また 計算 機 コンピュータ よる 知的 情報 処理 システム 設計 実現 関する 研究 分野 する 3 日本 大 百科 全書 ニッポニカ 解説 情報 工学 者 通信 工学 者 佐藤 理史 次 よう 述べる 1 人間 知的 能力 コンピュータ 上 実現 様々 技術 ソフトウェア コンピュータ システム 4 応用 例 自然 言語 処理 機械 翻訳 かな 漢字 変換 構文 解析 等 5 専門 家 推論 判断 模倣 エキスパート システム 画像 データ 解析 特定 パターン 検出 抽出 画像 認識 等 ある 4 1956 年 ダートマス 会議 ジョン マッカーシー よる 命名 現在 記号 処理 用いる 知能 記述 主体 する 情報 処理 研究 アプローチ いう 意味 あい 使う 家庭 用 電気 機械 器具 制御 システム ゲーム ソフト 思考 ルーチン こう 呼ぶ こと ある プログラミング 言語 LISP よる ELIZA いう カウンセラー 模倣 プログラム 人工 無脳 しばしば 引き合い 出す 計算 機 人間 専門 家 役割 する いう エキスパート システム 呼ぶ 研究 情報 処理 システム 実現 人間 暗黙 持つ 常識 記述 問題 なる 実用 利用 困難 視 人工 知能 実現 アプローチ する ファジィ 理論 ニューラル ネットワーク よう アプローチ 知る 従来 人工 知能 gofai good Old fashioned AI 差 記述 記号 明示 性 ある 後 サポート ベクター マシン 注目 集める また 自ら 経験 元 学習 行う 強化 学習 いう 手法 ある 宇宙 おく 知性 最も 強力 形質 レイ カーツ ワイル いう 言葉 通り 知性 機械 表現 実装 いう こと 極めて 重要 作業 2006 年 ディープ ラーニング 深層 学習 登場 2010 年 代 以降 ビッグ データ 登場 よる 一

### sudachiの設定を変えて使ってみる
https://www.anlp.jp/proceedings/annual_meeting/2019/pdf_dir/P8-5.pdf

In [ ]:
# Load SudachiPy with split mode C
cfg = {"split_mode": "C"}
nlp = Japanese(meta={"tokenizer": {"config": cfg}})

In [ ]:
# Sudachiで形態素解析し、分かち書き後のlemmaを取得する。

x_pos = ['SPACE', 'PUNCT', 'AUX', 'ADP', 'SYM', 'DET', 'SCONJ', 'PART']
tokens = list()
for line in lines:
  for token in nlp(line):
    pos = token.pos_
    if pos not in x_pos:
      #print(f'text:{token.text}, pos:{token.pos_}, tag:{token.tag_}, lemma:{token.lemma_}')
      tokens.append(token.lemma_)

In [ ]:
# すべてのlemmaを半角スペースでつないで、長い文字列にする。

doc_AI = ' '.join(tokens)
print(doc_AI)

人工知能 じん こうち 英 artificial intelligence AI エーアイ 計算 computation いう 概念 コンピュータ Computer いう 道具 用いる 知能 研究 計算機科学 computer science 一 分野 指す 語 1 言語 理解 推論 問題解決 知的 行動 人間 代わる コンピューター 行う 技術 2 また 計算機 コンピュータ よる 知的 情報処理 システム 設計 実現 関する 研究 分野 する 3 日本 大 百科全書 ニッポニカ 解説 情報 工学者 通信 工学者 佐藤 理史 次 よう 述べる 1 人間 知的 能力 コンピュータ 上 実現 様々 技術 ソフトウェア コンピュータシステム 4 応用 例 自然 言語 処理 機械 翻訳 かな 漢字 変換 構文 解析 等 5 専門家 推論 判断 模倣 エキスパートシステム 画像データ 解析 特定 パターン 検出 抽出 画像 認識 等 ある 4 1956 年 ダートマス 会議 ジョン マッカーシー よる 命名 現在 記号 処理 用いる 知能 記述 主体 する 情報処理 研究 アプローチ いう 意味あい 使う 家庭用電気機械器具 制御 システム ゲームソフト 思考 ルーチン こう 呼ぶ こと ある プログラミング 言語 LISP よる ELIZA いう カウンセラー 模倣 プログラム 人工 無脳 しばしば 引き合い 出す 計算機 人間 専門家 役割 する いう エキスパートシステム 呼ぶ 研究 情報処理 システム 実現 人間 暗黙 持つ 常識 記述 問題 なる 実用 利用 困難 視 人工的 知能 実現 アプローチ する ファジィ 理論 ニューラル ネットワーク よう アプローチ 知る 従来 人工知能 gofai good Old fashioned AI 差 記述 記号 明示 性 ある その後 サポート ベクター マシン 注目 集める また 自ら 経験 元 学習 行う 強化 学習 いう 手法 ある 宇宙 おく 知性 最も 強力 形質 レイ カーツ ワイル いう 言葉 通り 知性 機械的 表現 実装 いう こと 極めて 重要 作業 2006 年 ディープラーニング 深層学習 登場 2010 年 代 以降 ビッグデータ 登場 よる 一過性 流行 超える 社会 浸透 2016 年 2

### 上記の操作をまとめておこなう関数を定義しておく。
* 後で、各エントリについて、この関数を呼び出す。

In [ ]:
def morph(soup, nlp):

  lines = list()
  for para in soup.find_all('p'):
    lines.append(para.text)

  x_pos = ['SPACE', 'PUNCT', 'AUX', 'ADP', 'SYM', 'DET', 'SCONJ', 'PART']
  tokens = list()
  for line in lines:
    for token in nlp(line):
      pos = token.pos_
      if pos not in x_pos:
        tokens.append(token.lemma_)

  return ' '.join(tokens)

## 課題の手順(2)

### 「コンピュータ科学」の様々な分野に関するWikipediaエントリについて上記の作業を実行
*  「人工知能」エントリの下部にある「コンピュータ科学」の分野一覧から、aタグのhref属性にあるURLを抜き出す。
* ただし、__`/wiki/`__という文字列で始まっているURLであり、かつ、テンプレートの状態でないものだけを抜き出す。
 * Wikipediaのクローリングについてもっと詳しく知りたい場合は、下記のページ等を参照されたい。
   * https://medium.com/@robinlphood/tutorial-a-simple-crawler-for-wikipedia-d7b6f6f55d5

In [ ]:
# 「人工知能」エントリの下部にある「コンピュータ科学」の分野一覧から、aタグのhref属性にあるURLを抜き出す。

target_str = '表話編歴コンピュータ科学'
prefix = '/wiki/'

urls = dict()
for table in soup.find_all('table'):
  if table.text[:len(target_str)] != target_str: continue
  for td in table.find_all('td'):
    for a in td.find_all('a'):
      if not a.text: continue
      try:
        if a.text.find('英語版') == -1:
          href = a['href']
          if href[:len(prefix)] == prefix and href.find('/Template:') == -1 and href.find('/Category:') == -1:
            urls[a.text] = 'https://ja.wikipedia.org' + href
      except:
        continue

for k in urls:
  print(k, urls[k])

コンピュータ科学 https://ja.wikipedia.org/wiki/%E3%82%B3%E3%83%B3%E3%83%94%E3%83%A5%E3%83%BC%E3%82%BF%E7%A7%91%E5%AD%A6
ハードウェア https://ja.wikipedia.org/wiki/%E3%83%8F%E3%83%BC%E3%83%89%E3%82%A6%E3%82%A7%E3%82%A2
プリント基板 https://ja.wikipedia.org/wiki/%E3%83%97%E3%83%AA%E3%83%B3%E3%83%88%E5%9F%BA%E6%9D%BF
周辺機器 https://ja.wikipedia.org/wiki/%E5%91%A8%E8%BE%BA%E6%A9%9F%E5%99%A8
集積回路 https://ja.wikipedia.org/wiki/%E9%9B%86%E7%A9%8D%E5%9B%9E%E8%B7%AF
Systems on Chip (SoCs) https://ja.wikipedia.org/wiki/System-on-a-chip
エネルギー消費 (グリーン・コンピューティング) https://ja.wikipedia.org/wiki/%E3%82%B0%E3%83%AA%E3%83%BC%E3%83%B3IT
EDA https://ja.wikipedia.org/wiki/EDA
ハードウェアアクセラレーション https://ja.wikipedia.org/wiki/%E3%83%8F%E3%83%BC%E3%83%89%E3%82%A6%E3%82%A7%E3%82%A2%E3%82%A2%E3%82%AF%E3%82%BB%E3%83%A9%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3
コンピュータ・アーキテクチャ https://ja.wikipedia.org/wiki/%E3%82%B3%E3%83%B3%E3%83%94%E3%83%A5%E3%83%BC%E3%82%BF%E3%83%BB%E3%82%A2%E3%83%BC%E3%82%AD%E3%83%86%E3%82%AF%E3%83%81%E3%83%A3
組み込み

In [ ]:
# 各エントリをダウンロードしてparserのインスタンスを作り、辞書として保存。

soups = dict()
for k in urls:
  html = urlopen(urls[k]) 
  soups[k] = BeautifulSoup(html, 'html.parser')

In [ ]:
# 先ほど定義した関数morph()を使って各エントリを形態素解析し、lemmaが半角スペースで区切られた文字列へ変換する。

genre = list()
corpus = list()
for k in soups:
  genre.append(k)
  doc = morph(soups[k], nlp)
  corpus.append(doc)

In [ ]:
corpus

In [ ]:
# 再利用するために、csvファイルとして保存しておく。

import pandas as pd

genre.append('人工知能')
corpus.append(doc_AI)

df = pd.DataFrame(list(zip(genre, corpus)), columns=['genre', 'text'])
print(df.head())
df.to_csv('cs_corpus.csv')

      genre                                               text
0  コンピュータ科学  計算機科学 けい さん かがく 英 computer science コンピュータ 科学 情...
1    ハードウェア  ハードウェア 英語 hardware システム 物理的 構成要素 指す 一般 用語 日本語 ...
2    プリント基板  プリント基板 プリント きばる 短縮 形 pwb PCB 基板 一種 以下 ふた つ まとめ...
3      周辺機器  周辺 機器 しゅうへん きく また ペリフェラル 英 peripheral コンピュータ ゲ...
4      集積回路  集積回路 しゅう せき かいろ 英 integrated circuit IC 半導体 表面...


## 課題の手順(3)

### scikit-learnのCountVectorizerで単語の出現頻度を要素とするベクトルを作成
* これにより、各文書のベクトル表現が得られる。
* 興味のある対象のベクトル表現を得ることは、その対象を機械学習アルゴリズムの入力データとして使うための第一歩。

In [ ]:
# 先ほど作成したcsvファイルを読んで、textカラムをCountVectorizerのインスタンスでベクトル化する。
# ベクトルの要素は、各Wikipediaエントリにおける、各単語の出現回数になる。

from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('cs_corpus.csv')

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['text'])

In [ ]:
# CountVectorizerで得られるのはsparse matrixである。
print(X)

  (0, 9859)	16
  (0, 2521)	1
  (0, 2561)	1
  (0, 2481)	1
  (0, 716)	7
  (0, 1972)	5
  (0, 3567)	14
  (0, 9127)	12
  (0, 7527)	4
  (0, 9853)	25
  (0, 8848)	1
  (0, 6797)	1
  (0, 6437)	1
  (0, 7094)	1
  (0, 7484)	4
  (0, 10496)	1
  (0, 9075)	4
  (0, 6153)	12
  (0, 8329)	2
  (0, 2424)	12
  (0, 3568)	2
  (0, 2848)	4
  (0, 6273)	1
  (0, 10677)	5
  (0, 8847)	6
  :	:
  (122, 5425)	1
  (122, 6319)	1
  (122, 6001)	1
  (122, 9113)	1
  (122, 5547)	1
  (122, 84)	1
  (122, 85)	1
  (122, 138)	1
  (122, 86)	1
  (122, 87)	4
  (122, 9204)	1
  (122, 5717)	4
  (122, 6647)	1
  (122, 6957)	2
  (122, 8504)	1
  (122, 7267)	1
  (122, 5841)	1
  (122, 90)	1
  (122, 93)	1
  (122, 7158)	1
  (122, 2526)	1
  (122, 2459)	1
  (122, 8239)	1
  (122, 96)	1
  (122, 5566)	1


In [ ]:
# sparse matrixから、通常のNumPyの配列へ変換する。
X = X.toarray()
print('文書数:{}; 語彙サイズ：{}'.format(*X.shape))

文書数:123; 語彙サイズ：10771


In [ ]:
# CountVectorizerによって作られた語彙を取得する。
vocab = vectorizer.get_feature_names()
print(vocab)

In [ ]:
# コーパス全体での出現頻度順で上位20単語を見てみる。

import numpy as np

vocab = np.array(vocab)
print(vocab[np.argsort(- X.sum(axis=0))][:20])

['こと' 'ある' 'する' 'よる' 'いう' 'なる' 'もの' '言語' 'ため' 'コンピュータ' 'ソフトウェア' 'システム'
 '問題' '使う' '場合' '計算' 'ない' 'データ' 'それ' 'これ']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])

In [ ]:
X = X.toarray()

In [ ]:
X.shape

(123, 10771)

In [ ]:
X[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
# 「人工知能」エントリと他のエントリとの距離を求める。

from scipy.spatial import distance

genre = df['genre'].values.tolist()
index_AI = genre.index('人工知能')
print(f'「{genre[index_AI]}」と「{genre[10]}」との間での・・・')
print(f'ユークリッド距離: {np.linalg.norm(X[10] - X[index_AI])}')
print(f'内積: {np.dot(X[10], X[index_AI])}')
print(f'コサイン類似度に基づく距離: {distance.cosine(X[10], X[index_AI])}')

「人工知能」と「組み込みシステム」との間での・・・
ユークリッド距離: 1.3092280064956952
内積: 0.14296101350365437
コサイン類似度に基づく距離: 0.8570389864963457


In [ ]:
len(genre)

123

In [ ]:
for i in range(len(genre)):
  print(np.dot(X[i], X[index_AI]))

0.19972856451145343
0.07958257944205381
0.06932375217097608
0.06861721119422405
0.11543972992717506
0.07636263686034675
0.07698024146545353
0.0
0.09167901533035046
0.11651775957303973
0.14296101350365437
0.039695948496531436
0.06686261508626926
0.09457999881701312
0.058201622169880035
0.08160148993890329
0.0851338880659056
0.10035329638191715
0.10563653279681089
0.1298191658225303
0.10770923631130316
0.05226222075408618
0.14385023889712287
0.10254078155204015
0.10061827470016631
0.09807106727198683
0.07929321988980442
0.08840188429864658
0.06481001835518775
0.06740566604104804
0.041237804379967755
0.06835443300990743
0.14167177013886606
0.10063348165050318
0.08716150498070815
0.05485574991571685
0.05533785852365058
0.08240204074134677
0.12267661715612632
0.10788636695712009
0.10099429094178325
0.07181699820553371
0.100185596257048
0.11329909218137553
0.08921391023602457
0.13236406149176935
0.08530264947670979
0.08129334283583439
0.11203514528753422
0.09212779231436238
0.083172919477500

### ここから各自作業。
* 「人工知能」と、Wikipediaのエントリの類似度の上で、最も近いジャンルは、どれ？
* ユークリッド距離とコサイン類似度のうち、どちらを類似度として使った場合の答えが、より納得できる答えになっているか？

In [ ]:
genre

['コンピュータ科学',
 'ハードウェア',
 'プリント基板',
 '周辺機器',
 '集積回路',
 'Systems on Chip (SoCs)',
 'エネルギー消費 (グリーン・コンピューティング)',
 'EDA',
 'ハードウェアアクセラレーション',
 'コンピュータ・アーキテクチャ',
 '組み込みシステム',
 'リアルタイムシステム',
 'ディペンダビリティ',
 'ネットワーク',
 '通信プロトコル',
 'インタプリタ',
 'ミドルウェア',
 '仮想マシン',
 'オペレーティングシステム',
 'ソフトウェア品質',
 'ツール',
 'プログラミングパラダイム',
 'プログラミング言語',
 'コンパイラ',
 'ドメイン固有言語',
 'モデリング言語',
 'ソフトウェアフレームワーク',
 '統合開発環境',
 'ソフトウェア構成管理',
 'ソフトウェアライブラリ',
 'ソフトウェアリポジトリ',
 'ソフトウェア開発',
 'ソフトウェア開発プロセス',
 '要求分析',
 'ソフトウェア設計',
 'ソフトウェアデプロイメント',
 'ソフトウェアメンテナンス',
 'オープンソースモデル',
 '計算理論',
 '計算モデル',
 '形式言語',
 'オートマトン理論',
 '計算可能性理論',
 '計算複雑性理論',
 '意味論',
 'アルゴリズム',
 'アルゴリズム解析',
 '乱択アルゴリズム',
 '計算幾何学',
 '離散数学',
 '確率',
 '統計学',
 '数学ソフトウェア',
 '情報理論',
 '解析学',
 '数値解析',
 '情報システム',
 'データベース管理システム',
 '情報ストレージシステム',
 '企業情報システム',
 '地理情報システム',
 '意思決定支援システム',
 'プロセス制御システム',
 'データマイニング',
 '電子図書館',
 'コンピューティング・プラットフォーム',
 'デジタルマーケティング',
 'World Wide Web',
 '情報検索',
 'セキュリティ',
 '暗号理論',
 '形式手法',
 '侵入検知システム',
 'ネットワーク・セキュリティ',
 '情報セキュリティ',
 'ヒューマンコンピュータインタラクシ